# Game Summary

In [ ]:
import requests
from datetime import timedelta, date, datetime
import json
import pandas as pd
import os
import csv
import sys
import time

In [ ]:
sys.path.append('../../code')  # Replace '/path/to/code' with the actual path to your 'code' directory
from scrape_new import daterange, get_daily_game_ids

In [ ]:
def get_game_summary(api_key, game_id, delay_seconds=1):
    '''
    Fetches and extracts game summary information from the Sportradar API.

    Params:
    api_key: str - API key for accessing the Sportradar service
    game_id: str - Unique identifier for the game
    delay_seconds: int - Number of seconds to wait after the request is made

    Returns:
    game: dict - Dictionary containing formatted game summary if successful, None otherwise
    '''
    url = f'https://api.sportradar.com/mlb/trial/v7/en/games/{game_id}/summary.json'
    response = requests.get(url, params={'api_key': api_key})

    # Delay after making the request
    time.sleep(delay_seconds)

    # Initialize an empty dictionary to store game data
    game = {}

    # Check if the request was successful
    if response.status_code == 200:
        game_summary = response.json()
        
        # Accessing the 'game' key safely
        game_data = game_summary.get('game', {})
        
        # Extracting game details
        game['game_id'] = game_id
        game['away_team_abbr'] = game_data.get('away', {}).get('abbr', 'No abbreviation found')
        game['home_team_abbr'] = game_data.get('home', {}).get('abbr', 'No abbreviation found')
        
        # Handling scheduled time information
        scheduled_time = game_data.get('scheduled', 'No scheduled time found')
        game['date'] = scheduled_time.split('T')[0]
        game['start_time'] = scheduled_time.split('T')[1][:5] if 'T' in scheduled_time else 'Unknown'
        game['day_night'] = game_data.get('day_night', 'Unknown')

        # Extracting venue information
        venue_data = game_data.get('venue', {})
        game['venue'] = venue_data.get('name', 'No venue name found')
        game['field_type'] = venue_data.get('stadium_type', 'No stadium type found')

        # Statistics
        game['away_team_errors'] = game_data.get('away', {}).get('errors', 0)
        game['home_team_errors'] = game_data.get('home', {}).get('errors', 0)
        game['away_team_hits'] = game_data.get('away', {}).get('hits', 0)
        game['home_team_hits'] = game_data.get('home', {}).get('hits', 0)
        game['away_team_runs'] = game_data.get('away', {}).get('runs', 0)
        game['home_team_runs'] = game_data.get('home', {}).get('runs', 0)
        
        return game
    else:
        print(f"Failed to retrieve data for game ID {game_id}: Status Code {response.status_code}")
        return None

In [ ]:
# Define the function to save game summaries to a CSV file
def save_game_summaries_to_csv(game_summaries, filename):
    # Define the CSV fieldnames based on the keys in the game summary dictionary
    fieldnames = game_summaries[0].keys() if game_summaries else []
    
    # Write the game summaries to the CSV file
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(game_summaries)

In [ ]:
# Example usage
api_key = ''
start_date = datetime(2024, 3, 1).date()

all_game_summaries = []
for single_date in daterange(start_date):
    year = single_date.year
    month = f"{single_date.month:02d}"  # Format as two digits
    day = f"{single_date.day:02d}"     # Format as two digits
    game_ids = get_daily_game_ids(api_key, year, month, day)
    for game_id in game_ids:
        game_summary = get_game_summary(api_key, game_id,delay_seconds=1)
        if game_summary:
            all_game_summaries.append(game_summary)

# Save the game summaries to a CSV file
if all_game_summaries:
    filename = '../data/game_summaries-1.csv'
    for game_summary in all_game_summaries:
        df = pd.DataFrame([game_summary])
        df.to_csv(filename, mode='a', index=False, header=not os.path.exists(filename))
    print(f"Game summaries saved to {filename}")
else:
    print("No game summaries were collected.")